In [63]:
import geopy

In [64]:
geolocator = geopy.Nominatim()

In [65]:
location = geolocator.geocode('106 Corbett Avenue, SF')

In [66]:
location.latitude

37.7538681428571

In [67]:
import pandas as pd

In [68]:
data = pd.read_csv('./Eviction_Notices.csv')

/usr/local/lib/python3.4/site-packages/pandas/io/parsers.py:1164: DtypeWarning: Columns (4,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [69]:
data['days'] = pd.to_datetime(data['File Date'])

In [70]:
data.days = data.days.map(lambda x: (pd.to_datetime('2015-05-15') - x).days)

In [71]:
def clean(x):
    try:
        split = x[1:-1].split(',')
        return float(split[0])
    except Exception:
        pass

In [72]:
data['lat'] = data.Client_Location.map(lambda x: clean(x))

In [73]:
def clean(x):
    try:
        split = x[1:-1].split(',')
        return float(split[1])
    except Exception:
        pass

In [74]:
data['lon'] = data.Client_Location.map(lambda x: clean(x))

In [75]:
data = data[data.lon < 0]

In [76]:
data = data[data.lat > 0]

In [77]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32623 entries, 0 to 33240
Data columns (total 33 columns):
Eviction_ID                32623 non-null object
Address                    32623 non-null object
City                       32622 non-null object
State                      32623 non-null object
Zip                        31837 non-null object
File Date                  32623 non-null object
Non Payment                32623 non-null bool
Breach                     32623 non-null bool
Nuisance                   32623 non-null bool
Illegal Use                32623 non-null bool
Failure to Sign Renewal    32623 non-null bool
Access Denial              32623 non-null bool
Unapproved Subtenant       32623 non-null bool
Owner Move In              32623 non-null bool
Demolition                 32623 non-null bool
Capital Improvement        32623 non-null bool
Substantial Rehab          32623 non-null bool
Ellis Act WithDrawal       32623 non-null bool
Condo Conversion           32623 

In [78]:
data.columns

Index(['Eviction_ID', 'Address', 'City', 'State', 'Zip', 'File Date', 'Non Payment', 'Breach', 'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial', 'Unapproved Subtenant', 'Owner Move In', 'Demolition', 'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal', 'Condo Conversion', 'Roommate Same Unit', 'Other Cause', 'Late Payments', 'Lead Remediation', 'Development', 'Good Samaritan Ends', 'Constraints', 'Constraints Date', 'Supervisor_District', 'Neighborhood', 'Client_Location', 'days', 'lat', 'lon'], dtype='object')

In [79]:
clean = data.copy()

In [80]:
reasons = ['Non Payment', 'Breach', 'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial', 'Unapproved Subtenant', 'Owner Move In', 'Demolition', 'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal', 'Condo Conversion', 'Roommate Same Unit', 'Other Cause', 'Late Payments', 'Lead Remediation', 'Development']

In [81]:
x = data[reasons].stack()

In [82]:
data['reason'] = pd.Series(pd.Categorical(x[x!=0].index.get_level_values(1)))

In [83]:
df = data[['lat', 'lon', 'days', 'reason']]

In [84]:
df.head()

,lat,lon,days,reason
0,37.794134,-122.417956,45,Nuisance
1,37.764190,-122.420438,45,Non Payment
2,37.775926,-122.424579,45,Access Denial
3,37.773209,-122.402243,45,Other Cause
4,37.782804,-122.454488,45,Nuisance


In [85]:
df.reason.astype('object', replace_values=True)

TypeError: _astype() got an unexpected keyword argument 'replace_values'

In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import StandardScaler, Normalizer

scaler = StandardScaler()
n = Normalizer()

In [ ]:
df[['lat', 'lon', 'days']] = scaler.fit_transform(df[['lon', 'lat', 'days']])

In [ ]:
df.head()

In [ ]:
#df.to_json('evictions.json')

In [86]:
df.days.describe()

count    32623.000000
mean      3714.794685
std       2055.353269
min         45.000000
25%       1852.000000
50%       4008.000000
75%       5569.000000
max       6707.000000
Name: days, dtype: float64

In [90]:
data['File Date'][-1:]


33240    01/02/1997
Name: File Date, dtype: object